In [1]:
import numpy as np

In [2]:
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
count=CountVectorizer()

In [7]:
docs=np.array(['The Sun is shining',
               'The weather is sweet',
               'The sun is shining and the weahther is sweet'])

In [8]:
bag=count.fit_transform(docs)

In [9]:
print(count.vocabulary_)

{'the': 5, 'sun': 3, 'is': 1, 'shining': 2, 'weather': 7, 'sweet': 4, 'and': 0, 'weahther': 6}


In [10]:
print(bag.toarray())

[[0 1 1 1 0 1 0 0]
 [0 1 0 0 1 1 0 1]
 [1 2 1 1 1 2 1 0]]


In [11]:
count=CountVectorizer(ngram_range=(2,2))

In [12]:
bag=count.fit_transform(docs)

In [13]:
print(count.vocabulary_)

{'the sun': 5, 'sun is': 4, 'is shining': 1, 'the weather': 7, 'weather is': 9, 'is sweet': 2, 'shining and': 3, 'and the': 0, 'the weahther': 6, 'weahther is': 8}


In [14]:
print(bag.toarray())

[[0 1 0 0 1 1 0 0 0 0]
 [0 0 1 0 0 0 0 1 0 1]
 [1 1 1 1 1 1 1 0 1 0]]


In [15]:
#Processing of movie dataset

In [16]:
import pyprind 

In [17]:
import pandas as pd

In [18]:
import os

In [19]:
basepath='aclImdb'

In [20]:
labels={'pos':1,'neg':0}

In [21]:
pbar=pyprind.ProgBar(5000)

In [23]:
df=pd.DataFrame()

In [26]:
for s in ('test','train'):
    for l in ('pos','neg'):
        path=os.path.join(basepath,s,l)
        for file in os.listdir(path):
            with open(os.path.join(path,file),
                      'r',encoding='utf-8') as infile:
                txt=infile.read()
                df=df.append([[txt,labels[l]]],ignore_index=True)
                pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:05:42


In [27]:
df.columns=['review','sentiment']

In [28]:
import numpy as np

In [29]:
np.random.seed(0)

In [30]:
df=df.reindex(np.random.permutation(df.index))

In [31]:
df.to_csv('movie_data.csv',index=False,encoding='utf-8')

In [32]:
df=pd.read_csv('movie_data.csv',encoding='utf-8')

In [33]:
df.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


In [34]:
#Cleaning the text data

In [35]:
df.loc[0,'review'][-50:]

'is seven.<br /><br />Title (Brazil): Not Available'

In [36]:
import re

In [39]:
def preprocessor(text):
    text=re.sub('<[^>]*>','',text)
    emoticons=re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',text)
    text=(re.sub('[\W]+',' ',text.lower()) + ' '.join(emoticons).replace('-',''))
    return text

In [40]:
preprocessor(df.loc[0,'review'][-50:])

'is seven title brazil not available'

In [41]:
df['review']=df['review'].apply(preprocessor)

In [42]:
#Processing documents in token

In [43]:
def tokenizer(text):
    return text.split()

In [44]:
tokenizer('running like running and thus they run')

['running', 'like', 'running', 'and', 'thus', 'they', 'run']

In [16]:
from nltk.stem.porter import PorterStemmer

In [46]:
porter=PorterStemmer()

In [47]:
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [48]:
tokenizer_porter('running like running and thus they run')

['run', 'like', 'run', 'and', 'thu', 'they', 'run']

In [50]:
import nltk

In [51]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Satish\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [52]:
from nltk.corpus import stopwords

In [53]:
stop=stopwords.words('english')

In [55]:
[w for w in tokenizer_porter('running like running and thus they run') if w not in stop]

['run', 'like', 'run', 'thu', 'run']

In [56]:
X_train=df.loc[:250,'review'].values 

In [57]:
y_train=df.loc[:250,'sentiment'].values

In [58]:
X_test=df.loc[49000:,'review'].values

In [59]:
y_test=df.loc[49000:,'review'].values

In [60]:
from sklearn.model_selection import GridSearchCV

In [61]:
from sklearn.pipeline import Pipeline

In [62]:
from sklearn.linear_model import LogisticRegression

In [63]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [64]:
tfidf=TfidfVectorizer(strip_accents=None,
                      lowercase=False,
                      preprocessor=None
                     )

In [66]:
param_grid=[{'vect__ngram_range':[(1,1)],
             'vect__stop_words':[stop,None],
             'vect__tokenizer':[tokenizer,tokenizer_porter],
             'clf__penality':['l1','l2'],
             'clf__C':[1.0,10.0,100.0]},
           {'vect__ngram_range':[(1,1)],
             'vect_stop_words':[stop,None],
             'vect__tokenizer':[tokenizer,tokenizer_porter],
             'vect__use_idf':[False],
             'vect__norm':[None],
             'clf__penalty':['l1','l2'],
             'clf__C':[1.0,10.0,100.0]}]

In [68]:
lr_tfidf=Pipeline([('vect',tfidf),('clf',LogisticRegression(random_state=0))])

In [69]:
gs_lr_tfidf=GridSearchCV(lr_tfidf,param_grid,scoring='accuracy',cv=5,verbose=1,n_jobs=-1)

In [2]:
#Working with bigger dataset

In [46]:
import numpy as np
import re
from nltk.corpus import stopwords

In [47]:
stop=stopwords.words('english')

In [48]:
def tokenizer(text):
    text=re.sub('<[^>]*>','',text)
    emoticons=re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',text.lower())
    text=(re.sub('[\W]+',' ',text.lower()) + ' '.join(emoticons).replace('-',''))
    tokenized=[w for w in text.split() if w not in stop]
    return tokenized

In [49]:
def stream_docs(path):
    with open(path,'r',encoding='utf-8') as csv:
        next(csv)
        for line in csv:
            text,label=line[:-3],int(line[-2])
            yield text,label

In [50]:
next(stream_docs(path='movie_data.csv'))

('"In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />""Murder in Greenwich"" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich f

In [51]:
def get_minibatch(doc_stream,size):
    docs,y=[],[]
    try:
        for _ in range(size):
            text,label=next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None,None
    return docs,y

In [52]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

In [53]:
vect=HashingVectorizer(decode_error='ignore',n_features=2 **21,preprocessor=None,tokenizer=tokenizer)

In [54]:
clf=SGDClassifier(loss='log',random_state=1,n_iter=1)

In [55]:
doc_stream=stream_docs(path='movie_data.csv')

In [56]:
import pyprind 

In [57]:
pbar=pyprind.ProgBar(45)

In [58]:
classes=np.array([0,1])

In [59]:
for _ in range(45):
    X_train,y_train=get_minibatch(doc_stream,size=1000)
    if not X_train:
        break
    X_train=vect.transform(X_train)
    clf.partial_fit(X_train,y_train,classes=classes)
    pbar.update()

C:\Users\Satish\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
C:\Users\Satish\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
0% [#                             ] 100% | ETA: 00:01:49C:\Users\Satish\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
0% [##                            ] 100% | ETA: 00:01:26C:\Users\Satish\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_

C:\Users\Satish\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
0% [#####################         ] 100% | ETA: 00:00:15C:\Users\Satish\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
C:\Users\Satish\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
0% [######################        ] 100% | ETA: 00:00:12C:\Users\Satish\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_

In [60]:
X_test,y_test=get_minibatch(doc_stream,size=5000)

In [61]:
X_test=vect.transform(X_test)

In [62]:
print('Accuray %.3f' %clf.score(X_test,y_test))

Accuray 0.867


In [63]:
#LDA in scikit learn

In [64]:
import pandas as pd

In [65]:
df=pd.read_csv('movie_data.csv',encoding='utf-8')

In [66]:
from sklearn.feature_extraction.text import CountVectorizer

In [67]:
count=CountVectorizer(stop_words='english',max_df=.1,max_features=5000)

In [68]:
X=count.fit_transform(df['review'].values)

In [69]:
from sklearn.decomposition import LatentDirichletAllocation

In [70]:
lda=LatentDirichletAllocation(n_topics=10,random_state=123,learning_method='batch')

In [72]:
#X_topics=lda.fit_transform(X)